<a href="https://colab.research.google.com/github/djagatiya/Text2SQL/blob/main/Git_Text2SQL_ModelBuilding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets bitsandbytes accelerate peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (

In [29]:
import os
import torch
from datasets import Dataset, load_dataset
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

def set_deterministic(seed):
    # SET DETERMINISTIC
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True, warn_only=True)
    os.putenv("CUBLAS_WORKSPACE_CONFIG", ":4096:8")
    set_seed(seed, deterministic=True)

In [3]:
dataset = load_dataset("djagatiya/synthetic_text_to_sql_d14")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6713 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'sql_complexity', 'sql_prompt', 'sql_context', 'sql'],
        num_rows: 6713
    })
    test: Dataset({
        features: ['id', 'domain', 'sql_complexity', 'sql_prompt', 'sql_context', 'sql'],
        num_rows: 408
    })
})

In [4]:
dataset['train'][0]

{'id': 5131,
 'domain': 'healthcare',
 'sql_complexity': 'basic SQL',
 'sql_prompt': 'What is the minimum cultural competency score by worker?',
 'sql_context': 'CREATE TABLE worker_scores (worker_id INT, score INT); INSERT INTO worker_scores (worker_id, score) VALUES (1, 95), (2, 88), (3, 72);',
 'sql': 'SELECT worker_id, MIN(score) FROM worker_scores;'}

### Prepare Dataset

In [6]:
SQL_PROMPT = """
Instruct: Write SQL query of question asked by user based on following database structure context.
Context: {context}
Question: {question}
Output: {sql}
<|endoftext|>
"""

def prepare_dataset(sample):

    sample['text'] = SQL_PROMPT.format(
        context=sample['sql_context'],
        question=sample['sql_prompt'],
        sql=sample['sql']
    ).strip()

    return sample

In [7]:
dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/6713 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [11]:
print(dataset['train'][0]['text'])

Instruct: Write SQL query of question asked by user based on following database structure context.
Context: CREATE TABLE worker_scores (worker_id INT, score INT); INSERT INTO worker_scores (worker_id, score) VALUES (1, 95), (2, 88), (3, 72);
Question: What is the minimum cultural competency score by worker?
Output: SELECT worker_id, MIN(score) FROM worker_scores;
<|endoftext|>


In [17]:
model_name='microsoft/phi-2'

In [18]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    use_fast=False,
    trust_remote_code=True
)
tokenizer.add_special_tokens({'pad_token': '<|padding|>'})
print(tokenizer.all_special_tokens)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['<|endoftext|>', '<|padding|>']


In [19]:
encoded = tokenizer("Hello world<|endoftext|>", add_special_tokens=True, max_length=10, padding='max_length')
encoded

{'input_ids': [50295, 50295, 50295, 50295, 50295, 50295, 50295, 15496, 995, 50256], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]}

In [20]:
tokenizer.convert_ids_to_tokens(encoded['input_ids'], skip_special_tokens=False)

['<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 'Hello',
 'Ġworld',
 '<|endoftext|>']

In [28]:
train_max_len = max([len(tokenizer.encode(t)) for t in dataset['train']['text']])
train_max_len

534

In [60]:
# Free memory
import gc
if 'model' in globals():
    print("Existing model deleted.")
    del model
gc.collect()
torch.cuda.empty_cache()

# Set Seed
set_deterministic(1)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    quantization_config=bnb_config,
)

model.resize_token_embeddings(len(tokenizer))

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "mlp.fc1", "mlp.fc2"],
    modules_to_save=["lm_head", "embed_tokens"]
)

model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

Existing model deleted.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 330,966,136 || all params: 3,106,020,592 || trainable%: 10.6556


In [61]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import DataCollatorForLanguageModeling, TrainingArguments

In [62]:
response_template_with_context = "\nOutput:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)
response_template_ids

[198, 26410, 25]

In [63]:
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer,
    mlm=False
)

In [64]:
train_ds, test_ds = dataset['train'], dataset['test']

In [65]:
args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    # learning_rate=1e-4,
    optim="adamw_8bit",
    output_dir="model2",
    # save_embedding_layers=True,
    logging_strategy='steps',
    save_strategy='steps',
    evaluation_strategy='steps',
    eval_steps=300,
    logging_steps=300,
    save_steps=300,
    save_only_model=True
)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    data_collator=collator,
    args=args,
    dataset_text_field='text',
    train_dataset=train_ds,
    eval_dataset=test_ds,
    max_seq_length=600
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss,Validation Loss
300,0.197300,0.170346
600,0.189900,0.162319
900,0.177800,0.159404
1200,0.176000,0.154415


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Step,Training Loss,Validation Loss
300,0.197300,0.170346
600,0.189900,0.162319
900,0.177800,0.159404
1200,0.176000,0.154415
1500,0.160100,0.149537
1800,0.147500,0.145144
2100,0.138400,0.139313
2400,0.133900,0.135821
2700,0.131400,0.136104
3000,0.125800,0.135440


TrainOutput(global_step=3356, training_loss=0.15461336415487478, metrics={'train_runtime': 7930.59, 'train_samples_per_second': 1.693, 'train_steps_per_second': 0.423, 'total_flos': 4.043936056164211e+16, 'train_loss': 0.15461336415487478, 'epoch': 1.9997020706092656})

In [66]:
!zip -rj checkpoint-3300.zip /content/model2/checkpoint-3300

  adding: trainer_state.json (deflated 75%)
  adding: tokenizer_config.json (deflated 93%)
  adding: README.md (deflated 66%)
  adding: special_tokens_map.json (deflated 79%)
  adding: added_tokens.json (deflated 82%)
  adding: merges.txt (deflated 53%)
  adding: adapter_config.json (deflated 52%)
  adding: vocab.json (deflated 68%)
  adding: training_args.bin (deflated 51%)
  adding: adapter_model.safetensors (deflated 8%)


In [68]:
!cp /content/checkpoint-3300.zip /content/drive/MyDrive

In [12]:
import torch

In [67]:
torch.rand((10,5), device='cuda')

tensor([[0.6775, 0.3037, 0.6007, 0.7857, 0.4540],
        [0.2393, 0.2849, 0.0517, 0.4029, 0.6673],
        [0.0134, 0.9298, 0.7474, 0.4837, 0.1211],
        [0.9462, 0.7040, 0.6708, 0.0918, 0.2388],
        [0.8333, 0.7080, 0.7049, 0.8491, 0.6525],
        [0.1479, 0.9197, 0.8599, 0.6167, 1.0000],
        [0.2291, 0.6571, 0.4803, 0.2039, 0.1653],
        [0.4140, 0.0778, 0.8636, 0.7247, 0.9803],
        [0.4414, 0.5709, 0.0962, 0.7754, 0.6884],
        [0.9774, 0.7150, 0.3359, 0.0247, 0.8986]], device='cuda:0')